In [1]:
import jieba
import os
import re
import numpy as np
import jieba.posseg as psg
import networkx as nx
import pandas as pd
import math

In [2]:
# -*- coding:utf-8 -*-
import pymysql

# ******************************************数据库操作***************************************************#
class Database:
    # 数据库初始化，port得是数值int型,暂时先这样，（后续情况需要可以开放多库多服务器查询（允许传入参数）
    def __init__(self, ):
        self.host = '127.0.0.1'  # 主机名，默认本地
        # self.host = '101.201.198.35'  # 主机名，为服务器ip
        self.port = 3306  # 端口默认3306
        self.user = 'root'  # 用户名，默认root
        self.password = '13886139062wang'  # 密码
        # self.password = 'STourism@whu'  # 服务器密码
        self.db = 'zhihu'  # 数据库名
        self.charset = 'utf8'  # 数据库字符集，默认utf-8,7.31 xts,修改为utf8mb4，支持存储表情

    # **********************************数据库连接并执行的相关函数*********************************************#2.12更新可以批量插入和查询一个
    def database(self, sql, fetch='all', data_list='one'):
        o_type = sql.split(' ')[0].lower()
        conn = pymysql.connect(host=self.host, port=self.port, user=self.user, password=self.password, db=self.db, charset=self.charset)  # 连接数据库
        cur = conn.cursor()  # 用于访问和操作数据库中的数据（一个游标，像一个指针）
        if o_type == 'select':
            cur.execute(sql)  # 执行操作
            if fetch == 'all':
                result = cur.fetchall()  # 匹配所有满足的
            else:
                result = cur.fetchone()  # 匹配一个满足的
            cur.close()  # 关闭游标
            conn.close()  # 关闭数据库连接
            return result
        elif o_type == 'insert' or o_type == 'update' or o_type == 'delete':
            if data_list == 'one':
                try:
                    cur.execute(sql)
                    conn.commit()  # 提交事务
                    # print("{} ok".format(type))
                except Exception as e:  # 发生错误时回滚
                    print(e)
                    conn.rollback()  # 回滚事务
            else:  # 一次插入多条数据
                try:
                    cur.executemany(sql, data_list)  # 列表里面的单个数据必须是元组
                    conn.commit()  # 提交事务
                    # print("{} ok".format(type))
                except Exception as e:  # 发生错误时回滚
                    print(e)
                    conn.rollback()  # 回滚事务
            cur.close()  # 关闭游标
            conn.close()  # 关闭数据库连接


if __name__ == '__main__':
    Data_admin = Database()

In [3]:
def get_stop_dict(file):
    content = open(file,encoding="utf-8")
    word_list = []
    for c in content:
        c = re.sub('\n|\r','',c)
        word_list.append(c)
    return word_list

In [4]:
def get_data(path):
    t = open(path,encoding="utf-8")
    data = t.read()
    t.close()
    return data

In [20]:
def get_wordlist(text,maxn,synonym_words,stop_words):
    synonym_origin = list(synonym_words['origin'])
    synonym_new = list(synonym_words['new'])
    flag_list = ['n','nz','vn']#a,形容词，v,形容词
    counts={}
    
    new_text = ''

    for index in text.index:
        index_text = text.loc[index].values[0][:10]
        new_text = new_text + index_text + '\n'
    
    print(new_text)

    text_seg = psg.cut(new_text)
    for word_flag in text_seg:
        #word = re.sub("[^\u4e00-\u9fa5]","",word_flag.word)
        word = word_flag.word
        if word_flag.flag in flag_list and len(word)>1 and word not in stop_words:
            if word in synonym_origin:
                index = synonym_origin.index(word)
                word = synonym_new[index]
            counts[word]=counts.get(word,0)+1
            
    
    words= sorted(counts.items(),key=lambda x:x[1],reverse=True)
    words= list(dict(words).keys())[0:maxn]
    
    return words

In [18]:
def get_t_seg(topwords,text,synonym_words,stop_words):
    word_docs = {}
    synonym_origin = list(synonym_words['origin'])
    synonym_new = list(synonym_words['new'])
    flag_list = ['n','nz','vn']#a,形容词，v,形容词
    
    text_lines_seg =[]

    for index in text.index:
        t_seg = []
        text_seg = psg.cut(text.loc[index].values[0][:10])
        for word_flag in text_seg:
            # word = re.sub("[^\u4e00-\u9fa5]","",word_flag.word)
            word = word_flag.word
            if word_flag.flag in flag_list and len(word)>1 and word not in stop_words:
                if word in synonym_origin:
                    word = synonym_new[synonym_origin.index(word)]
                if word in topwords:
                    t_seg.append(word)
                    
        t_seg = list(set(t_seg))
        # print(t_seg)
        for word in t_seg:
            word_docs[word] = word_docs.get(word,0)+1
        text_lines_seg.append(t_seg)
    return text_lines_seg,word_docs

In [7]:
def get_comatrix(text_lines_seg):
    comatrix = pd.DataFrame(np.zeros([len(topwords),len(topwords)]),columns=topwords,index=topwords)
    for t_seg in text_lines_seg:
        for i in range(len(t_seg)-1):
                for j in range(i+1,len(t_seg)):
                    comatrix.loc[t_seg[i],t_seg[j]]+=1
    for k in range(len(comatrix)):
        comatrix.iloc[k,k]=0
    return comatrix

In [8]:
def get_pmi(word1,word2,word_docs,co_matrix,n):
    pw1 = word_docs[word1]/n
    pw2 = word_docs[word2]/n
    pw1w2 = (co_matrix.loc[word1][word2]+co_matrix.loc[word2][word1])/n
    if pw1w2/(pw1*pw2)<=0:
        return 0
    else:
        pmi = math.log2(pw1w2/(pw1*pw2))
        return pmi

In [9]:
def get_net(copmi,topwords):
    g = nx.Graph()
    for i in range(len(topwords)-1):
        word = topwords[i]
        for j in range(i+1,len(topwords)):
            w=0
            word2 = topwords[j]
            w = copmi.loc[word][word2]+copmi.loc[word2][word]
            if w>0:
                g.add_edge(word,word2,weight=w)
    return g

In [10]:
#文件路径
dic_file = "./stop_dic/dict.txt"
stop_file = "./stop_dic/stopwords.txt"
synonym_file = "./stop_dic/synonym_list.xlsx"

In [11]:
import pandas as pd

sql = 'select answer_text from zhihu_movie_cleaned'
raw_Data = Data_admin.database(sql)
data = pd.DataFrame(raw_Data)

data

,0
0,一个字，爱。\n两个字，强推。\n三个字，炒鸡棒。\n四个字，表白全员。\n五个字，藕饼一生...
1,当时去电影院看的，感觉些许漫长的同时又觉得还可以！开始觉得哪吒的形象很丑，但是看着看着觉得丑...
2,我不懂，只是作为普通观众单纯的提的问题，为什么包括影中哪吒敖丙，还有别的国漫例如孙悟空。在所...
3,是近年来的优秀国漫之一，除了人物如刘梓晨一般的脸奇丑无比以外，当时去电影院看的时候真心觉得敖...
4,在人物塑造上很成功。不论是出于内疚而循循诱导，寓教于乐的太乙，还是身负使命的敖丙，亦或是满怀...
...,...
30864,为自己热爱的事业奋斗终生
30865,工作才有退休，事业没有。
30866,退休了还要来知乎答题 拍电影就不算什么了吧
30867,Great Question! Because when you are doing wha...


In [12]:
#读取文件
stop_words = get_stop_dict(stop_file)
jieba.load_userdict(dic_file)
synonym_words = pd.read_excel(synonym_file)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/6d/2k7_g1z929x_ljmzs94gg_kw0000gn/T/jieba.cache
Loading model cost 0.444 seconds.
Prefix dict has been built successfully.


In [21]:
#数据处理
n_topwords=200
topwords = get_wordlist(data,n_topwords,synonym_words,stop_words)

一个字，爱。
两个字当时去电影院看的，感我不懂，只是作为普通是近年来的优秀国漫之在人物塑造上很成功。借、鉴、了电影《阿童《哪咤》体现了中国哲风格比较喜欢，这种美精彩的都已经在电影预没事别玩泥巴！看完之后，不关注造型《哪吒之魔童降世》
至于他们一直讨论的哪说实话我不喜欢这部作嘿嘿，电影的动作指导小哪吒魔珠投胎，一出一、前情
特别喜欢2如果进行街头采访，相当年看完这电影，心里更像是一部科普育儿知不知不觉距离上映已经「你不是哪吒，你不够今天偶然又看了一遍，体现了中国动漫电影的态度可以效果不错
结《哪吒之魔童降世》上怎么讲呢，哪吒的模型片子本身不错，一家人昨晚在电脑又重温了一很好看，蛮有意思的，7.27早上自己来到接近现实德不配位。镜头的转换看过，最感人的话就是哪吒就一畜生，400最近刚补完《哪吒》
哪吒这部电影一反常态一部内核三观有问题，。其实，现在再回头仔细被捧成金子的真石漆礅在电影里我看到了太多特效不错，剧情就算了——相信这几天很多人的朋首先在此不得不去承认姜子牙师尊为了一统三《哪吒》票房36亿，不喜欢，并且没有任何推荐下这个回答，讲的说实话，我个人感觉挺泪目
魔丸一开始也不事实证明国产观众低龄这个电影跟之前的哪吒优点：引起了儿子对中今天晚上跟WQ看了期很好的动画片，很多细很普通，全程面无表情在山河社稷图中那一段有笑点，有泪点，无尿今日影评禾隹与野兔
222说实话 我觉得挺难看还记得去年那个夏天，如果是好莱坞的动画电国漫加油一般 除了画面很美，垃圾。
首先梗玩的太推荐指数：☆☆☆☆☆每次看到电视里有放，这部电影，我看了三遍哪吒这部动画电影的评何其相似，我只能感慨标准的合家欢、爆米花很难做自己，但是我愿我觉得，那些遭遇过社冲呀
关于哪吒的影视正片结束时哪吒的眼睛焚琴煮鹤，匪夷所思
戏说不是胡说，改编不就你，也配叫哪吒？也国漫因为市场原因一直看第一遍的时候没看懂⬇️⬇️⬇️⬇️

看完以后觉得，我命由这将是我第一次在知乎剧情一般，四年级小学2010年10月16君君臣臣父父子子，
资源不存在
《哪吒之2020看到了还是想好看我没看过
因为我觉着老套的东西，也就一群一般吧，我给5分。笑细节做的很好画质还可，立意还行，关于角色形象设计得很《哪吒》里申公豹有一不请自来，还是迟到已本来十分反感这个人物有剧透！有剧透！有剧没看到，但是现在越来很多人说看了《哪吒之我看这个电影的时候从现

In [19]:
t_segs,word_docs = get_t_seg(topwords,data,synonym_words,stop_words)
n = len(t_segs)
print(n)
co_matrix = get_comatrix(t_segs)

擦破
30869


In [15]:
copmi = pd.DataFrame(np.zeros([len(topwords),len(topwords)]),columns=topwords,index=topwords)
for i in range(len(topwords)-1):
    word1 = topwords[i]
    for j in range(i+1,len(topwords)):
        word2 = topwords[j]
        copmi[word1][word2] = get_pmi(word1,word2,word_docs,co_matrix,n)

In [16]:
co_net =get_net(copmi,topwords)

In [17]:
nx.write_gexf(co_net,"./result/word_pmi4.gexf")